In [125]:
import numpy as np
import pandas as pd
import json
import sklearn
from sklearn.preprocessing import LabelEncoder 
from pandas.io.json import json_normalize
import seaborn as sns
sns.set_style("whitegrid")
from pandas.plotting import scatter_matrix

### Useful Sections:
Regression Notebook 3 "Why add Qualitative Data to Regression"

Regression Notebook 3 "How to Choose the Best Model" - k-fold cross validation

Unsupervised Learning Notebook 1 - k-Means clustering

# Cleaning

In [109]:
bot_labels = pd.read_csv('data/botwiki-2019.tsv', names = ['id', 'identity'], sep = '\t')
human_labels = pd.read_csv('data/verified-2019.tsv', names = ['id', 'identity'], sep = '\t')
bot_labels['bot'] = 1
human_labels['bot'] = 0
#IMPROVE?
# First set all beers as 0
#beers_train['Stout'] = 0
# Then locate all the stouts and set Stout to 1
#beers_train.loc[beers_train.Beer_Type == "Stout",'Stout'] = 1

In [110]:
#if IOUP data rate exceeded: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

#convert json to DataFrame
file = open('data/botwiki-2019_tweets.json')
file1 = open('data/verified-2019_tweets.json')
jsn = json.load(file)
jsn1 = json.load(file1)
bot_accts = pd.DataFrame(pd.json_normalize(jsn))
human_accts = pd.DataFrame(pd.json_normalize(jsn1))

In [111]:
#merge users and labels
bot_accts = bot_accts.merge(bot_labels, left_on=bot_accts['user.id'], right_on=bot_labels['id'])
human_accts = human_accts.merge(human_labels, left_on=human_accts['user.id'], right_on=human_labels['id'])
accts = bot_accts.append(human_accts)
#accts.to_csv('bot_wiki_verified_19.csv', index = False)

In [112]:
#filters accts to only numerical variables for simple knn

accts_knn = accts.filter(items = ['user.id', 'user.followers_count', 'user.friends_count', 'user.listed_count', 'user.favourites_count', 
                    'bot'])

accts_knn.tail()

,user.id,user.followers_count,user.friends_count,user.listed_count,user.favourites_count,bot
1982,76235237,3020,790,18,1707,0
1983,2159329092,4074,4239,163,4777,0
1984,25527618,17014,2610,608,7765,0
1985,43654274,367516,20245,5470,10282,0
1986,19084210,55512,11725,1366,2429,0


# Loading Data

In [176]:
accts = pd.read_csv('data/Cumulative_raw_data.csv')

In [177]:
pd.set_option('display.max_colwidth', None) # prevents descriptions from being cut off

In [178]:
accts['description'] = accts['description'].values.astype('U') #formating as unicode for Tfidf

In [130]:
accts = accts[:10000] #shortening for exploratory data work to make easier on computer
print(len(accts))

10000


In [179]:
#One hot encoding bot
accts['bot'] = 0
accts.loc[accts.species == "bot",'bot'] = 1
accts.head()

,Unnamed: 0,follow_request_sent,has_extended_profile,profile_use_background_image,default_profile_image,profile_background_image_url_https,verified,translator_type,profile_text_color,profile_image_url_https,...,created_at,contributors_enabled,time_zone,protected,default_profile,is_translator,species,source,withheld_in_countries,bot
0,602249341,False,False,False,False,https://abs.twimg.com/images/themes/theme4/bg.gif,False,none,000000,https://pbs.twimg.com/profile_images/923924342974578688/k5RCrlSQ_normal.jpg,...,Thu Jun 07 22:16:27 +0000 2012,False,London,False,False,False,human,botometer-feedback-2019,NaN,0
1,797617218511060992,False,True,False,False,https://abs.twimg.com/images/themes/theme1/bg.png,False,none,000000,https://pbs.twimg.com/profile_images/855244571697061888/YDpSuAXy_normal.jpg,...,Sun Nov 13 01:48:58 +0000 2016,False,Pacific Time (US & Canada),False,False,False,bot,botometer-feedback-2019,NaN,1
2,889925474,False,False,True,False,https://pbs.twimg.com/profile_background_images/723350044/881da62975b5e0dc69fc5903a094c9df.jpeg,False,none,333333,https://pbs.twimg.com/profile_images/964079832295288832/WSnP8xQ-_normal.jpg,...,Thu Oct 18 23:19:38 +0000 2012,False,NaN,False,False,False,human,botometer-feedback-2019,NaN,0
3,96435556,False,True,True,False,https://abs.twimg.com/images/themes/theme6/bg.gif,False,none,333333,https://pbs.twimg.com/profile_images/3114299697/f73cd7211520f13e8121d4ccd5db2481_normal.jpeg,...,Sat Dec 12 22:53:04 +0000 2009,False,Rome,False,False,False,bot,botometer-feedback-2019,NaN,1
4,16905397,False,False,True,False,https://pbs.twimg.com/profile_background_images/3248110/DSCF4726.jpg,False,regular,666666,https://pbs.twimg.com/profile_images/969705141644410880/MJasCoOk_normal.jpg,...,Wed Oct 22 13:43:42 +0000 2008,False,Bern,False,False,False,human,botometer-feedback-2019,NaN,0


# Transmuting Text to Numbers

In [230]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [233]:
text = open('sample.txt', 'w', encoding = 'utf-8')

In [234]:
vectorizer = CountVectorizer()
for desc in accts['description']:
       text.write(desc.strip().replace('\n+', ''))
text.close

<function TextIOWrapper.close()>

In [235]:
text = open('sample.txt', 'r', encoding = 'utf-8')

In [236]:
vectorizer.fit(text)
#print(vectorizer.vocabulary_)


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [240]:
#for i in range(len(accts['description'])):
#    text = accts['description'][i]
#    accts['user.description'][i] = vectorizer.transform(text)
#accts['desc_vectorized'] = vectorizer.transform(accts['description'])
#accts['desc_vectorized'] = accts['desc_vectorized'].toarray()

### TfidfVectorizer

In [273]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [274]:
#accts_copy = accts.copy()
#accts_train = accts_copy.sample(frac = 0.75, random_state = 1400)
#accts_test = accts_copy.drop(accts_train.index)

In [275]:
y = accts[['bot']].to_numpy()
X = accts[['description']]
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.25, random_state=1400)
accts_tfidf_train = accts_knn.sample(frac = 0.75,  random_state = 876)

In [276]:
vectorizer = TfidfVectorizer()

In [277]:
#print(accts['user.description'])
vectorizer.fit(X_train['description'].tolist())

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [278]:
print(vectorizer.vocabulary_)

{'cosplay': 12300, 'fan': 17663, 'animal': 5603, 'lover': 28576, 'check': 10656, 'out': 34381, 'https': 22572, 'co': 11348, 'qhleoa4grb': 37637, 'pretty': 36672, 'girl': 20163, 'gamer': 19605, 'health': 21705, 'come': 11562, 'on': 33916, 'yrn5zjdyug': 51539, 'god': 20429, 'gives': 20203, 'his': 22074, 'hardest': 21473, 'battles': 7420, 'to': 45618, 'strongest': 43336, 'soldiers': 42404, 'you': 51445, 'love': 28559, 'it': 24390, 'harshly': 21511, 'in': 23536, 'caxiscy5q4': 10242, 'how': 22457, 'do': 14737, 'like': 28019, 'my': 31651, 'site': 41900, 'ylg9eimkcv': 51331, 'me': 29899, 'sqchbhs1w0': 42889, 'fitness': 18355, 'lp7advslf2': 28613, 'jugador': 25637, 'de': 13557, 'fútbol': 19447, 'profesional': 36841, 'actualmente': 4313, 'club': 11263, 'deportivo': 13879, 'toluca': 45691, 'edk2sp0lhg': 15741, 'instagram': 23902, 'sambueza_1984': 40432, 'nan': 31928, 'phd': 35632, 'candidate': 9912, 'fu_berlin': 19173, 'visiting': 48426, 'scholar': 40757, 'princeton': 36729, 'former': 18780, 'co

In [279]:
print(vectorizer.idf_)

[ 8.85276802  8.52734562 10.72457019 ... 10.72457019 10.72457019
 10.72457019]


In [280]:
X_train = vectorizer.transform(X_train['description'])
#vector = vector.toarray()
#print(vector.reshape(-1,1))

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(X_train, y_train.ravel())

In [282]:
X_test = vectorizer.transform(X_test['description'])
y_predict = knn.predict(X_test)
print("Our model has a ",
      np.round(sum(y_predict == y_test.ravel())/len(y_test)*100,2),
      "% accuracy on the testing set")

Our model has a  74.41 % accuracy on the testing set


### Hashing

In [329]:
from sklearn.feature_extraction.text import HashingVectorizer

In [330]:
y = accts[['bot']].to_numpy()
X = accts[['description']]
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.25, random_state=1400)
accts_tfidf_train = accts_knn.sample(frac = 0.75,  random_state = 876)

In [331]:
h_vectorizer = HashingVectorizer(n_features = 20)

In [332]:
X_train = h_vectorizer.transform(X_train['description'])

In [333]:
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(X_train, y_train.ravel())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [334]:
X_test = h_vectorizer.transform(X_test['description'])
y_predict = knn.predict(X_test)
print("Our model has a ",
      np.round(sum(y_predict == y_test.ravel())/len(y_test)*100,2),
      "% accuracy on the testing set")

Our model has a  77.67 % accuracy on the testing set


### CountVectorizer

In [350]:
from sklearn.feature_extraction.text import CountVectorizer

In [351]:
y = accts[['bot']].to_numpy()
X = accts[['description']]
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.25, random_state=1400)
accts_tfidf_train = accts_knn.sample(frac = 0.75,  random_state = 876)

In [352]:
c_vectorizer = CountVectorizer()

In [353]:
c_vectorizer.fit(X_train['description'].tolist())

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [354]:
X_train = c_vectorizer.transform(X_train['description'])

In [355]:
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(X_train, y_train.ravel())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [356]:
X_test = c_vectorizer.transform(X_test['description'])
y_predict = knn.predict(X_test)
print("Our model has a ",
      np.round(sum(y_predict == y_test.ravel())/len(y_test)*100,2),
      "% accuracy on the testing set")

Our model has a  80.44 % accuracy on the testing set


### Combining TfidfVectorization, Hashing, CountVectorization

# K Nearest Neighbors

In [222]:
#K-fold cross validation
from sklearn.model_selection import KFold
from sklearn.base import clone

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [223]:
#Creating test train split for knn
X = accts[['followers_count', 'friends_count', 'listed_count', 'favourites_count', ]].to_numpy()
y = accts[['bot']].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.25, random_state=1400)
accts_knn_train = accts_knn.sample(frac = 0.75,  random_state = 876)

In [224]:
knn = KNeighborsClassifier(n_neighbors = 10)

In [225]:
knn.fit(X_train, y_train.ravel())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [226]:
y_predict = knn.predict(X_test)

In [227]:
print("Our model has a ",
      np.round(sum(y_predict == y_test.ravel())/len(y_test)*100,2),
      "% accuracy on the testing set")

Our model has a  88.01 % accuracy on the testing set


# K-means Clustering

In [27]:
from sklearn.cluster import KMeans

In [28]:
kmeans = KMeans(2)

In [29]:
kmeans.fit(accts_train)

ValueError: could not convert string to float: 'bot'

In [ ]:
#Test Train Split
